In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
from datetime import datetime
import re
from pyproj import Proj, transform
from geopy.geocoders import Nominatim
from tqdm import tqdm
import requests
from urllib.parse import urlparse
import json

In [70]:
sign = pd.read_csv('./data/표지판 최종.csv')
light = pd.read_csv('./data/신호등 찐 최종.csv')
camera = pd.read_csv('./data/camera.csv')
df_1 = pd.read_csv('./data/구별그룹화.csv')
df_2 = pd.read_csv('./data/동별그룹화.csv')

In [71]:
sign.head()

,Unnamed: 0,0,구,동
0,0,서울특별시 중구 순화동,중구,순화동
1,1,서울특별시 종로구 소격동,종로구,소격동
2,2,서울특별시 성북구 성북동,성북구,성북동
3,3,서울특별시 성북구 성북동,성북구,성북동
4,4,서울특별시 성북구 성북동,성북구,성북동


In [72]:
light.head()

,Unnamed: 0,구,동
0,0,서초구,내곡동
1,1,서대문구,북가좌동
2,2,은평구,응암동
3,3,은평구,응암동
4,4,송파구,잠실동


In [26]:
camera.head()

,제한속도,구,동
0,50,중랑구,신내동
1,30,도봉구,방학동
2,50,노원구,상계동
3,50,서대문구,창천동
4,50,중구,을지로6가


In [27]:
sign = sign[['구','동']]
sign.head()

,구,동
0,중구,순화동
1,종로구,소격동
2,성북구,성북동
3,성북구,성북동
4,성북구,성북동


In [28]:
light = light[['구','동']]
light.head()

,구,동
0,서초구,내곡동
1,서대문구,북가좌동
2,은평구,응암동
3,은평구,응암동
4,송파구,잠실동


In [33]:
df_1

,시군구,정상시평균속도,전체차량주행거리,승용차주행거리,버스주행거리,화물차주행거리,전체차량추정교통량,승용차추정교통량,버스추정교통량,화물차추정교통량,차량평균속도,차량혼잡빈도강도,차량혼잡시간강도,혼잡시평균속도
0,강남구,46,333427,270363,3903,59161,16200,13151,182,2866,25,52,73,16
1,강동구,45,190956,153058,3059,34839,12864,10484,155,2224,26,52,69,17
2,강북구,40,89268,75805,278,13184,9538,8098,29,1410,22,49,70,14
3,강서구,47,222298,174870,3536,43891,12690,9989,159,2541,25,51,68,16
4,관악구,44,96911,76636,626,19649,11969,9435,77,2457,24,36,60,15
5,광진구,54,177010,138375,2134,36500,15905,12443,185,3276,26,68,81,19
6,구로구,43,185863,141887,1796,42179,10829,8269,109,2449,22,50,69,15
7,금천구,44,206110,154599,1949,49561,12296,9285,101,2909,23,55,72,16
8,노원구,43,184258,146879,1768,35610,11107,8911,96,2099,23,56,73,16
9,도봉구,39,92167,78325,328,13512,8478,7229,27,1220,22,48,68,14


In [30]:
len(df_2)

423

In [39]:
light_group = light.groupby('구').count()
light_group=light_group.reset_index()
light_group

,구,동
0,강남구,6611
1,강동구,4324
2,강북구,3158
3,강서구,6083
4,관악구,3550
5,광진구,3120
6,구로구,4349
7,금천구,2665
8,노원구,6562
9,도봉구,3235


In [43]:
df_1['신호등'] = light_group['동']

In [44]:
df_1

,시군구,정상시평균속도,전체차량주행거리,승용차주행거리,버스주행거리,화물차주행거리,전체차량추정교통량,승용차추정교통량,버스추정교통량,화물차추정교통량,차량평균속도,차량혼잡빈도강도,차량혼잡시간강도,혼잡시평균속도,신호등
0,강남구,46,333427,270363,3903,59161,16200,13151,182,2866,25,52,73,16,6611
1,강동구,45,190956,153058,3059,34839,12864,10484,155,2224,26,52,69,17,4324
2,강북구,40,89268,75805,278,13184,9538,8098,29,1410,22,49,70,14,3158
3,강서구,47,222298,174870,3536,43891,12690,9989,159,2541,25,51,68,16,6083
4,관악구,44,96911,76636,626,19649,11969,9435,77,2457,24,36,60,15,3550
5,광진구,54,177010,138375,2134,36500,15905,12443,185,3276,26,68,81,19,3120
6,구로구,43,185863,141887,1796,42179,10829,8269,109,2449,22,50,69,15,4349
7,금천구,44,206110,154599,1949,49561,12296,9285,101,2909,23,55,72,16,2665
8,노원구,43,184258,146879,1768,35610,11107,8911,96,2099,23,56,73,16,6562
9,도봉구,39,92167,78325,328,13512,8478,7229,27,1220,22,48,68,14,3235


In [46]:
sign_group = sign.groupby('구').count().reset_index()
sign_group

,구,동
0,강남구,21307
1,강동구,16197
2,강북구,13256
3,강서구,17343
4,관악구,15407
5,광진구,13630
6,구로구,13264
7,금천구,8161
8,노원구,11788
9,도봉구,8090


In [67]:
df_1

,시군구,정상시평균속도,전체차량주행거리,승용차주행거리,버스주행거리,화물차주행거리,전체차량추정교통량,승용차추정교통량,버스추정교통량,화물차추정교통량,차량평균속도,차량혼잡빈도강도,차량혼잡시간강도,혼잡시평균속도,신호등,표지판
0,강남구,46,333427,270363,3903,59161,16200,13151,182,2866,25,52,73,16,6611,21307
1,강동구,45,190956,153058,3059,34839,12864,10484,155,2224,26,52,69,17,4324,16197
2,강북구,40,89268,75805,278,13184,9538,8098,29,1410,22,49,70,14,3158,13256
3,강서구,47,222298,174870,3536,43891,12690,9989,159,2541,25,51,68,16,6083,17343
4,관악구,44,96911,76636,626,19649,11969,9435,77,2457,24,36,60,15,3550,15407
5,광진구,54,177010,138375,2134,36500,15905,12443,185,3276,26,68,81,19,3120,13630
6,구로구,43,185863,141887,1796,42179,10829,8269,109,2449,22,50,69,15,4349,13264
7,금천구,44,206110,154599,1949,49561,12296,9285,101,2909,23,55,72,16,2665,8161
8,노원구,43,184258,146879,1768,35610,11107,8911,96,2099,23,56,73,16,6562,11788
9,도봉구,39,92167,78325,328,13512,8478,7229,27,1220,22,48,68,14,3235,8090


In [68]:
df_1.to_csv('./data/구별 전처리 완료.csv', encoding='utf-8-sig', index=False)

In [69]:
df_2

,시군구,읍면동,정상시평균속도,전체차량주행거리,승용차주행거리,버스주행거리,화물차주행거리,전체차량추정교통량,승용차추정교통량,버스추정교통량,화물차추정교통량,차량평균속도,차량혼잡빈도강도,차량혼잡시간강도,혼잡시평균속도
0,종로구,사직동,44.90,241666,190804,1392,49470,10560,8338,61,2162,23.51,84.42,92.26,17.41
1,종로구,삼청동,36.85,68407,53824,324,14259,7983,6281,38,1664,24.67,37.95,57.03,16.67
2,종로구,부암동,46.06,252474,200247,1371,50856,19050,15110,103,3837,32.92,60.27,73.98,19.90
3,종로구,평창동,49.78,438158,344403,2701,91054,12002,9434,74,2494,34.47,51.61,64.48,18.45
4,종로구,무악동,45.29,38347,30788,204,7355,19545,15692,104,3749,23.93,37.40,68.53,13.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,강동구,둔촌2동,43.48,153975,128414,1200,24362,16830,14036,131,2663,30.25,24.42,50.91,15.85
419,강동구,암사1동,40.83,146755,125891,670,20193,9451,8107,43,1300,24.37,34.70,58.20,15.14
420,강동구,천호2동,40.92,37497,32394,111,4992,6874,5938,20,915,21.02,56.32,77.88,14.17
421,강동구,길동,50.83,234535,187268,3149,44117,15085,12045,203,2838,27.90,45.37,62.16,20.06
